In [1]:
import json
import pandas as pd

# JSON 파일 로드
with open("./so_data/model_test_data.json", "r", encoding="utf-8") as file: # test용 실제 기사 json형식 파일
    data = json.load(file)

# 데이터프레임으로 변환
df_test = pd.DataFrame(data)

# 데이터 확인
print(df_test.head())

# --------
# --------

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 피처 생성
df_test['title_length'] = df_test['title'].apply(lambda x: len(x.split()))
df_test['content_length'] = df_test['content'].apply(lambda x: len(x.split()))
df_test['title_content_ratio'] = df_test['title_length'] / (df_test['content_length'] + 1e-6)
df_test['special_char_count'] = df_test['title'].apply(lambda x: sum([1 for char in x if char in '!?#']))
df_test['number_in_title'] = df_test['title'].apply(lambda x: 1 if any(char.isdigit() for char in x) else 0)

# TF-IDF 유사도 계산
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix_title = tfidf_vectorizer.fit_transform(df_test['title'])
tfidf_matrix_content = tfidf_vectorizer.transform(df_test['content'])

cosine_similarities = np.array([
    np.dot(tfidf_matrix_title[i].toarray(), tfidf_matrix_content[i].toarray().T)[0, 0]
    for i in range(len(df_test))
])
df_test['tfidf_similarity'] = cosine_similarities

# --------
# --------

X_test = df_test[['content_length', 'title_content_ratio', 'tfidf_similarity']]  # features

# 스케일링 적용
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# --------
# --------

import joblib

# 모델 불러오기
rf_model = joblib.load("random_forest_model.pkl")  # 학습한 모델이 저장된 파일 경로 입력

# 예측 확률 수행
y_pred_proba = rf_model.predict_proba(X_test_scaled)

# 결과 출력
for title, proba in zip(df_test['title'], y_pred_proba):
    clickbait_proba = proba[1]  # Clickbait 클래스(1)의 확률
    normal_proba = proba[0]     # Normal 클래스(0)의 확률
    
    # 확률이 높은 클래스를 예측 결과로 설정
    label = "Clickbait" if clickbait_proba > normal_proba else "Normal"
    
    # 출력
    print(f"제목: {title}")
    print(f"예측 결과: {label}")
    print(f"Clickbait 확률: {clickbait_proba:.2f}, Normal 확률: {normal_proba:.2f}\n")


                                 title  \
0  송리단길서 불에 탄 포르쉐...차주는 물 한 잔 붓고 나 몰라라   
1     가짜뉴스 범벅 '극단적' 유튜브, 왜 똑똑한 사람도 홀릴까   

                                             content  
0  대형 항공사 승무원이 불 붙은 차량을 카페 주차장에 방치한 채 떠나 큰 화재로 이어...  
1  평균보다 교육 수준이 낮고 인지력이 떨어지는 사람이 '가짜 뉴스'에 현혹될 확률이 ...  
제목: 송리단길서 불에 탄 포르쉐...차주는 물 한 잔 붓고 나 몰라라
예측 결과: Clickbait
Clickbait 확률: 0.67, Normal 확률: 0.33

제목: 가짜뉴스 범벅 '극단적' 유튜브, 왜 똑똑한 사람도 홀릴까
예측 결과: Normal
Clickbait 확률: 0.22, Normal 확률: 0.78

